In [1]:
from sqlalchemy import create_engine
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import pandas as pd

In [2]:
username = 'michals2'
password = 'Nv1VZRmuTxRvJn3u'
host = 'mysql.agh.edu.pl'
port = 3306
database = 'michals2'

connection_string = f'mysql+pymysql://{username}:{password}@{host}:{port}/{database}'

In [3]:
engine = create_engine(connection_string, echo=True)

In [6]:
table_name = 'Perfumes'

df = pd.read_sql_table(table_name, con=engine)
data_list = df.to_dict(orient='records')  # Each row as a dictionary in a list

2025-04-16 22:40:02,082 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2025-04-16 22:40:02,083 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-04-16 22:40:02,202 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2025-04-16 22:40:02,203 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-04-16 22:40:02,265 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2025-04-16 22:40:02,266 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-04-16 22:40:02,455 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-04-16 22:40:02,458 INFO sqlalchemy.engine.Engine DESCRIBE `michals2`.`Perfumes`
2025-04-16 22:40:02,459 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-04-16 22:40:02,514 INFO sqlalchemy.engine.Engine SHOW FULL TABLES FROM `michals2`
2025-04-16 22:40:02,515 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-04-16 22:40:02,577 INFO sqlalchemy.engine.Engine SHOW FULL TABLES FROM `michals2`
2025-04-16 22:40:02,578 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-04-16 22:40:02,651 INFO sqlalchem

df #use as dataframe </br>
data_list #use as dict

define user prompt

In [7]:
user_prompt = "I like woody, tabacco notes which would be perfect for evening."

content based filtering

In [8]:
df['text_features'] = df['Description'].fillna('') + ' ' + df['Accords'].fillna('') + ' ' + df['Designer'].fillna('') + ' ' + df['TopNotes'].fillna('') + ' ' + df['MiddleNotes'].fillna('') + ' ' + df['BaseNotes'].fillna('')

In [9]:
vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = vectorizer.fit_transform(df['text_features'])

user_vec = vectorizer.transform([user_prompt])

cos_similarities = cosine_similarity(user_vec, tfidf_matrix).flatten()

In [10]:
top_indices = cos_similarities.argsort()[::-1][:5]
recommended_perfumes = df.iloc[top_indices][['Name', 'Designer', 'Description', 'Accords', 'TopNotes', 'MiddleNotes', 'BaseNotes']]
recommended_perfumes['similarity'] = cos_similarities[top_indices]

In [11]:
print("Top perfume recommendations:")
recommended_perfumes

Top perfume recommendations:


,Name,Designer,Description,Accords,TopNotes,MiddleNotes,BaseNotes,similarity
12,Chilling By The Pacific,The Dua Brand,Chilling By The Pacific by The Dua Brand is a ...,citrus (100%); aromatic (83.5618%); green (59....,"['Citron', 'Coriander', 'Mint', 'Apricot', 'Or...",[],[],0.077640
1098,Dia Woman,Amouage,Dia Woman by Amouage is a Floral fragrance for...,floral (100%); fresh (99.8436%); aldehydic (88...,"['Aldehydes', 'Cyclamen', 'Violet Leaf', 'Sage...","['Peony', 'Turkish Rose Oil', 'Orris Root', 'O...","['White Musk', 'Heliotrope', 'Sandalwood', 'In...",0.076541
52,Sand Desert At Sunset,Zara,Sand Desert At Sunset by Zara is a Oriental Sp...,warm spicy (100%); cinnamon (76.7838%); amber ...,"['Orange', 'Incense']","['Cinnamon', 'Tonka Bean', 'Chocolate']","['Iris', 'Cedar']",0.075917
644,Cinéma,Yves Saint Laurent,Cinéma by Yves Saint Laurent is a Oriental Flo...,floral (100%); vanilla (75.8754%); amber (73.5...,"['Almond Blossom', 'Clementine', 'Cyclamen']","['Jasmine', 'Peony', 'Amaryllis']","['Vanilla', 'Amber', 'Benzoin', 'White Musk']",0.071787
0,Unicorn Milkshake,The Dua Brand,Unicorn Milkshake by The Dua Brand is a Floral...,sweet (100%); vanilla (58.1656%); lactonic (55...,[],[],[],0.071374


In [17]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('all-MiniLM-L6-v2')
text_features = df['text_features'].tolist()

embeddings = model.encode(text_features, convert_to_tensor=True)
user_embedding = model.encode(user_prompt, convert_to_tensor=True)

cos_scores = util.cos_sim(user_embedding, embeddings).cpu().numpy().flatten()
top_indices = cos_scores.argsort()[::-1][:5]

In [16]:
recommended_perfumes = df.iloc[top_indices][['Name', 'Designer', 'Description', 'Accords', 'TopNotes', 'MiddleNotes', 'BaseNotes']]
recommended_perfumes['similarity'] = cos_similarities[top_indices]

# Display
recommended_perfumes

,Name,Designer,Description,Accords,TopNotes,MiddleNotes,BaseNotes,similarity
998,,By Kilian,,woody (100%); powdery (70.7588%); fruity (62.3...,"['Bergamot', 'Lemon']","['Fig Nectar', 'Green Notes', 'Hedione']","['Vanilla', 'Sandalwood', 'Cedar', 'Orris', 'A...",0.015269
950,,Mancera,,coffee (100%); woody (92.2342%); warm spicy (7...,"['Black Currant', 'Bergamot', 'Peach']","['Coffee', 'Amber', 'Floral Notes']","['Woody Notes', 'Sweet Notes', 'White Musk']",0.032694
766,More Than Words,Xerjoff,,amber (100%); oud (68.8718%); fruity (64.8205%...,"['Agarwood (Oud)', 'Fruity Notes', 'Floral Not...",[],[],0.040287
450,Dangerous Lily,Dzintars,,white floral (100%); floral (76.6668%); woody ...,"['Lily', 'Floral Notes', 'dark woodsy notes', ...",[],[],0.026576
977,,By Kilian,,sweet (100%); woody (75.308%); fruity (67.0148...,"['Cognac', 'Raspberry', 'Liquor']","['Tonka Bean', 'Bulgarian Rose', 'Oakmoss']","['Praline', 'Oak', 'Sandalwood']",0.007445
